After some tweaking to the lambda api calls, plus based on the experiments I got from v1, now try v2 of analysis

In [1]:
import numpy as np
import pandas as pd
import pymongo
from pymongo import MongoClient
client = MongoClient()

from pprint import pprint
from bson.son import SON
#for geo parsing
import json
from shapely.geometry import shape, Point
import geopandas as gpd
from shapely.ops import cascaded_union

In [2]:
import datetime

In [3]:
#db setup-- the posts collection is the only one for stashing everything returned from that api endpoint
#note to self-- use mongoDatabase2
db = client['instagram-london']
coll = db.posts

In [4]:
#to illustrate what one of these entries llok like
coll.find_one({'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}})

{'_id': ObjectId('57e808b5e6b178079dc2c447'),
 'attribution': None,
 'caption': {'created_time': '1474822692',
  'from': {'full_name': 'Marco Perotti',
   'id': '221313401',
   'profile_picture': 'https://scontent.cdninstagram.com/t51.2885-19/10354293_749726858382492_1377793777_a.jpg',
   'username': '_marcoperotti_'},
  'id': '17863046944025263',
  'text': '#coffeetime in #littlevenice ☕️ #regentscanal #london'},
 'comments': {'count': 0},
 'created_time': '1474822692',
 'filter': 'Normal',
 'id': '1347232848809173922_221313401',
 'images': {'low_resolution': {'height': 320,
   'url': 'https://scontent.cdninstagram.com/t51.2885-15/s320x320/e35/14262817_628480587358803_8893996333568884736_n.jpg?ig_cache_key=MTM0NzIzMjg0ODgwOTE3MzkyMg%3D%3D.2.l',
   'width': 320},
  'standard_resolution': {'height': 640,
   'url': 'https://scontent.cdninstagram.com/t51.2885-15/s640x640/sh0.08/e35/14262817_628480587358803_8893996333568884736_n.jpg?ig_cache_key=MTM0NzIzMjg0ODgwOTE3MzkyMg%3D%3D.2.l',
   'w

In [5]:
#since I changed the range of the lat lngs returned from the lambda function, what I want is all the data after ~ 4th Sept 2016
rawInstagramPosts = coll.find({'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}})

In [6]:
uniqueLocationsCursor = db.posts.aggregate([
        {"$match": {'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}}},
        {"$group": { "_id": {"locationid": "$location.id","lat": "$location.latitude", "lng": "$location.longitude" }, 
                    "count": { "$sum": 1 }  }}
    ]);

uniqueLocations = list(uniqueLocationsCursor)

In [7]:
len(uniqueLocations)

61938

In [27]:
#now use Geopandas to parse the locations into the london census output areas
londonGeoDF = gpd.read_file('OA_2011_BGC_london.json')
#now trim the londonGeo stuff to may it smaller... hopefully this will make any op faster
londonDFSubset = londonGeoDF.ix[:,['OA11CD', 'POPDEN', 'WD11NM_BF', 'geometry']]

In [29]:
#using shapely, construct a geodataframe from the uniqueLocations
postLocationpoints = [Point(x['_id']['lng'], x['_id']['lat']) for x in uniqueLocations]
locationsDF = pd.DataFrame({'geometry': postLocationpoints})
locationsDF['locationCount'] = pointCount
locationsDF['locationID'] = [x['_id']['locationid'] for x in uniqueLocations]
locationsDF = gpd.GeoDataFrame(locationsDF)

#then add lat lng fields in the df for non Shapely ops
locationsDF['lng'] = locationsDF['geometry'].map(lambda x: x.xy[0][0])
locationsDF['lat'] = locationsDF['geometry'].map(lambda x: x.xy[1][0])

NameError: name 'pointCount' is not defined

In [9]:
cursor =  db.posts.find({'$and': [{'timestamp': {'$gte': datetime.datetime(2016, 9, 10, 0,0,0)}},
                         {'timestamp':{'$lt': datetime.datetime(2016, 9, 11, 0, 0, 0)}}, 
                         {'type': "image"}]}, 
               projection=['location','timestamp','likes.count','images.low_resolution','tags'])

In [10]:
oneDayData = pd.DataFrame(list(cursor))

In [12]:
oneDayData[:15]

,_id,images,likes,location,tags,timestamp
0,57d88608d285908f0e1429af,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 44},"{'longitude': -0.23990477333333, 'latitude': 5...","[minimal_perfection, jj_indetail, igersoftheda...",2016-09-10 23:59:55
1,57d88608d285908f0e1424e2,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 29},"{'longitude': -0.3183405110173, 'latitude': 51...",[],2016-09-10 23:59:50
2,57d885f9d285908f0e13c821,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 19},"{'longitude': 0.14031792365121, 'latitude': 51...",[],2016-09-10 23:59:46
3,57d885f5d285908f0e13a7d0,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 4},"{'longitude': 0.11606, 'latitude': 51.49525, '...",[],2016-09-10 23:59:36
4,57d88602d285908f0e1401d2,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 1},"{'longitude': -0.185, 'latitude': 51.583, 'id'...","[praisdentalcare, colgatebrightsmile, colgateb...",2016-09-10 23:59:33
5,57d88601d285908f0e13fa28,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 18},"{'longitude': -0.16990783261964, 'latitude': 5...","[londonliving, streetphotography, love, nighto...",2016-09-10 23:59:08
6,57d88608d285908f0e14292b,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 3},"{'longitude': -0.25005279108905, 'latitude': 5...","[djs, popup, foodie, nw10, platta, music]",2016-09-10 23:59:08
7,57d885fad285908f0e13c9b2,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 10},"{'longitude': -0.0003945, 'latitude': 51.61669...","[nomakeup, nofilter, needsomemakeupforfil, lov...",2016-09-10 23:59:07
8,57d88606d285908f0e141a2b,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 14},"{'longitude': -0.22422306363769, 'latitude': 5...","[saturdaynight, awesome, drink, tbt, partytime...",2016-09-10 23:58:56
9,57d885f7d285908f0e13b39e,"{'low_resolution': {'width': 320, 'height': 32...",{'count': 20},"{'longitude': -0.50962436, 'latitude': 51.4288...","[crazygirl, thropepark, london]",2016-09-10 23:58:52
